## Creating custom model

This notebook describes ways to implement your own model.

Typical use cases:
1. creating models which can not be built by "block models builders" from ENOT framework;
1. you already have your model, and you don't want to rewrite your code.

### Main chapters of this notebook:
1. Setup the environment
1. Prepare dataset and create dataloaders
1. Create model and move it into search space
1. Check pretrain, search and tune phases

## Setup the environment
First, let's set up the environment and make some common imports.

In [ ]:
import os

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# You may need to change this variable to match free GPU index
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from pathlib import Path

import torch
import torch.nn as nn

from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_optimizer import RAdam

from enot.models import SearchSpaceModel
from enot.models.mobilenet import MobileNetBaseHead
from enot.models.mobilenet import MobileNetBaseStem
from enot.models.operations import SearchableMobileInvertedBottleneck
from enot.models.operations import SearchableFuseableSkipConv
from enot.models.operations import SearchVariantsContainer
from enot.optimize import EnotPretrainOptimizer
from enot.optimize import EnotSearchOptimizer

from tutorial_utils.train import accuracy
from tutorial_utils.train import WarmupScheduler

from tutorial_utils.checkpoints import download_getting_started_pretrain_checkpoint
from tutorial_utils.dataset import create_imagenette_dataloaders
from tutorial_utils.phases import tutorial_pretrain_loop
from tutorial_utils.phases import tutorial_search_loop
from tutorial_utils.phases import tutorial_train_loop

### In the following cell we setup all necessary dirs

* `ENOT_HOME_DIR` - ENOT framework home directory
* `ENOT_DATASETS_DIR` - root directory for datasets (imagenette2, ...)
* `PROJECT_DIR` - project directory to save training logs, checkpoints, ...

In [ ]:
ENOT_HOME_DIR = Path.home() / '.enot'
ENOT_DATASETS_DIR = ENOT_HOME_DIR / 'datasets'
PROJECT_DIR = ENOT_HOME_DIR / 'custom_model'

ENOT_HOME_DIR.mkdir(exist_ok=True)
ENOT_DATASETS_DIR.mkdir(exist_ok=True)
PROJECT_DIR.mkdir(exist_ok=True)

## Prepare dataset and create dataloaders

In [ ]:
dataloaders = create_imagenette_dataloaders(
    dataset_root_dir=ENOT_DATASETS_DIR, 
    project_dir=PROJECT_DIR,
    input_size=(224, 224),
    batch_size=32,
)

## Create model and move it into search space
To create your own model, you should build it as normal PyTorch model, following these rules:
1. Search options must be placed into `SearchVariantsContainer` module;
1. All `SearchVariantsContainer`'s must have the same number of operations. Operations in different containers can be different. **This restriction will be removed in the future versions.**
1. You can add any PyTorch operations into `SearchVariantsContainer`, if you do not need to account latency. Otherwise, you should use predefined oprations from `enot.model.operations`, or extend your operation from `LatencyMixin` (see "Tutorial - using latency optimization").

In [ ]:
# defining your model class
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.stem = MobileNetBaseStem(
            in_channels=3
        )
        self.body = nn.ModuleList([
            # 3 blocks with 3 search options in each block 
            self.build_search_variants_1(16, 24, 2),
            self.build_search_variants_1(24, 24, 1),
            self.build_search_variants_1(24, 24, 1),
            # 2 fixed blocks
            self.build_mib_k3_e6(24, 32, 2),
            self.build_mib_k3_e6(32, 32, 1),
            # 3 blocks with 3 search options in each block
            self.build_search_variants_1(32, 64, 2),
            self.build_search_variants_1(64, 64, 1),
            self.build_search_variants_1(64, 64, 1),
            # 1 fixed block
            self.build_mib_k3_e6(64, 96, 1),
            # 2 blocks with 3 search options in each block
            self.build_search_variants_2(96, 160, 2),
            self.build_search_variants_2(160, 160, 1),
            # 1 block with 3 search options
            self.build_search_variants_2(160, 320, 1),
        ])
        self.head = MobileNetBaseHead(
            bottleneck_channels=320,
            last_channels=1280, 
            num_classes=10,
        )

    @staticmethod
    def build_search_variants_1(in_channels, out_channels, stride):
        return SearchVariantsContainer([
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=stride,
                expand_ratio=6,
            ),
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=5,
                stride=stride,
                expand_ratio=6,
            ),
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=7,
                stride=stride,
                expand_ratio=3,
            ),
        ])

    @staticmethod
    def build_search_variants_2(in_channels, out_channels, stride):
        return SearchVariantsContainer([
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=stride,
                expand_ratio=6,
            ),
            SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=5,
                stride=stride,
                expand_ratio=6,
            ),
            SearchableFuseableSkipConv(
                in_channels=in_channels,
                out_channels=out_channels,
                stride=stride,
            ),
        ])

    @staticmethod
    def build_mib_k3_e6(in_channels, out_channels, stride):
        return SearchableMobileInvertedBottleneck(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=stride,
                expand_ratio=6,
            )

    def forward(self, x):
        x = self.stem(x)

        for block in self.body:
            x = block(x)

        x = self.head(x)

        return x

In [ ]:
model = MyModel()

# move model to search space
search_space = SearchSpaceModel(model).cuda()

## Check pretrain, search and tune phases

In this tutorial we use the same pretrain/search/train loops as in "Tutorial - getting started".

**IMPORTANT:**<br>
We set `N_EPOCHS`= 3 in this example to make tutorial execution faster. This is not enough for good pretrain quality, and you should set `N_EPOCHS`>= 100 if you want to achieve good results.

In [ ]:
N_EPOCHS = 3
N_WARMUP_EPOCHS = 1
len_train = len(dataloaders['pretrain_train_dataloader'])

optimizer = SGD(params=search_space.model_parameters(), lr=0.06, momentum=0.9, weight_decay=1e-4)
enot_optimizer = EnotPretrainOptimizer(search_space=search_space, optimizer=optimizer)
scheduler = CosineAnnealingLR(optimizer, T_max=len_train*N_EPOCHS, eta_min=1e-8)
scheduler = WarmupScheduler(scheduler, warmup_steps=len_train*N_WARMUP_EPOCHS)
loss_function = nn.CrossEntropyLoss().cuda()

tutorial_pretrain_loop(
    epochs=N_EPOCHS,
    search_space=search_space,
    enot_optimizer=enot_optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['pretrain_train_dataloader'],
    validation_loader=dataloaders['pretrain_validation_dataloader'],
    scheduler=scheduler,
)

In [ ]:
optimizer = RAdam(search_space.architecture_parameters(), lr=0.01)
enot_optimizer = EnotSearchOptimizer(search_space=search_space, optimizer=optimizer)

tutorial_search_loop(
    epochs=5,
    search_space=search_space,
    enot_optimizer=enot_optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['search_train_dataloader'],
    validation_loader=dataloaders['search_validation_dataloader'],
    latency_loss_weight=1e-3,
    latency_type='mmac',
    scheduler=None,
)

In [ ]:
# get regular model with the best architecture
best_model = search_space.get_network_with_best_arch().cuda()

In [ ]:
optimizer = RAdam(best_model.parameters(), lr=5e-3, weight_decay=4e-5)

tutorial_train_loop(
    epochs=5,
    model=best_model,
    optimizer=optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['tune_train_dataloader'],
    validation_loader=dataloaders['tune_validation_dataloader'],
    scheduler=None,
)